<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB24_BdD_Vectorielles_FAISS_et_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Vector Database Setup with FAISS and Pinecone

**Objectif:** Créer des embeddings, les indexer avec FAISS localement, puis les déployer sur une base vectorielle managée (Pinecone) pour un usage de niveau production.

**Durée estimée:** 20–30 minutes

**Livrable:** Scripts/notebook qui construisent, interrogent et persistent un index FAISS ; avec en plus un index Pinecone fonctionnel avec upsert/query.

---

## Table des matières
1. Installation et configuration de l'environnement
2. Corpus de documents exemple
3. Génération des embeddings (OpenAI)
4. FAISS: construction, recherche, persistance (local)
5. Pinecone: création d'index, upsert, query (managé)
6. (Optionnel) Intégration LangChain avec Pinecone
7. Prochaines étapes et expérimentations

## 1. Installation et Configuration de l'Environnement

⏱️ **Temps estimé:** 5–10 minutes

In [1]:
# Installation des dépendances
!pip install -q openai tiktoken faiss-cpu pinecone-client langchain langchain-openai langchain-pinecone chromadb python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/5

### Configuration des clés API

**Option 1 (Recommandée pour Colab):** Utiliser les Secrets de Colab
- Cliquez sur l'icône 🔑 dans la barre latérale gauche
- Ajoutez `OPENAI_API_KEY` et `PINECONE_API_KEY`
- Activez l'accès au notebook

**Option 2:** Définir directement dans le code (⚠️ Ne pas partager le notebook publiquement)

In [3]:
import os
from google.colab import userdata

# Option 1: Récupérer depuis les Secrets Colab
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')
    print("✅ Clés API chargées depuis les Secrets Colab")
except Exception as e:
    print(f"⚠️ Secrets non trouvés: {e}")
    print("Veuillez configurer manuellement ci-dessous...\n")

    # Option 2: Configuration manuelle (décommenter et remplir)
    # os.environ["OPENAI_API_KEY"] = "sk-..."  # Votre clé OpenAI
    # os.environ["PINECONE_API_KEY"] = "..."   # Votre clé Pinecone

# Configuration du nom de l'index Pinecone
os.environ["PINECONE_INDEX"] = "agentic-ai-lab"

# Vérification
if "OPENAI_API_KEY" in os.environ and "PINECONE_API_KEY" in os.environ:
    print("✅ Configuration complète")
    print(f"   - Index Pinecone: {os.environ['PINECONE_INDEX']}")
else:
    print("❌ Configuration incomplète - veuillez définir vos clés API")

✅ Clés API chargées depuis les Secrets Colab
✅ Configuration complète
   - Index Pinecone: agentic-ai-lab


## 2. Ensemble de documents servant d'exemple

Nous utilisons un petit ensemble de données sur l'IA agentique.

On pourra le remplacer par d'autres données plus tard.

In [4]:
# Documents exemple
docs = [
    {"id": "d1", "text": "Agentic AI agents use tools, memory, and goals to act."},
    {"id": "d2", "text": "LangChain and CrewAI help orchestrate multi-agent workflows."},
    {"id": "d3", "text": "RAG retrieves external knowledge to improve answer accuracy."},
    {"id": "d4", "text": "Vector databases enable fast similarity search over embeddings."},
    {"id": "d5", "text": "Planning loops and ReAct improve reasoning in complex tasks."},
]

# Requêtes de test
queries = [
    "How do agents use memory?",
    "Name a framework for multi-agent orchestration.",
    "Why is RAG useful?"
]

print(f"📚 Corpus: {len(docs)} documents")
print(f"🔍 Requêtes: {len(queries)} questions\n")

for doc in docs:
    print(f"  [{doc['id']}] {doc['text']}")

📚 Corpus: 5 documents
🔍 Requêtes: 3 questions

  [d1] Agentic AI agents use tools, memory, and goals to act.
  [d2] LangChain and CrewAI help orchestrate multi-agent workflows.
  [d3] RAG retrieves external knowledge to improve answer accuracy.
  [d4] Vector databases enable fast similarity search over embeddings.
  [d5] Planning loops and ReAct improve reasoning in complex tasks.


## 3. Génération des Embeddings (OpenAI)

Nous utilisons le modèle `text-embedding-3-small` d'OpenAI (rapide, 1536 dimensions).

In [5]:
from openai import OpenAI

# Initialisation du client OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
EMB_MODEL = "text-embedding-3-small"  # Bon compromis performance/coût, 1536-d

def get_embedding(text: str):
    """Génère un embedding pour un texte donné."""
    response = client.embeddings.create(model=EMB_MODEL, input=text)
    return response.data[0].embedding

print("🔄 Génération des embeddings...\n")

# Embeddings des documents
X = [get_embedding(d["text"]) for d in docs]
print(f"✅ Documents encodés: {len(X)} vecteurs")

# Embeddings des requêtes
Q = [get_embedding(q) for q in queries]
print(f"✅ Requêtes encodées: {len(Q)} vecteurs")

# Dimension des vecteurs
dim = len(X[0])
print(f"\n📊 Dimension des embeddings: {dim}")

🔄 Génération des embeddings...

✅ Documents encodés: 5 vecteurs
✅ Requêtes encodées: 3 vecteurs

📊 Dimension des embeddings: 1536


## 4. FAISS: construction, recherche, persistance (en local)

FAISS (Facebook AI Similarity Search) est une bibliothèque optimisée pour la recherche de similarité.

### 4a. Construction de l'index FAISS

In [6]:
import faiss
import numpy as np

# Conversion en array NumPy
xb = np.array(X, dtype="float32")

# Création d'un index avec produit scalaire (Inner Product)
# Note: normalisation des vecteurs pour que IP ≈ similarité cosinus
index = faiss.IndexFlatIP(dim)

# Normalisation L2 des vecteurs (pour similarité cosinus)
faiss.normalize_L2(xb)

# Ajout des vecteurs à l'index
index.add(xb)

print(f"✅ Index FAISS créé")
print(f"📊 Vecteurs indexés: {index.ntotal}")
print(f"📏 Dimension: {index.d}")

✅ Index FAISS créé
📊 Vecteurs indexés: 5
📏 Dimension: 1536


### 4b. Recherche dans l'index FAISS

In [7]:
def faiss_search(query_vec, k=3):
    """
    Recherche les k vecteurs les plus similaires.

    Args:
        query_vec: Vecteur de requête
        k: Nombre de résultats à retourner

    Returns:
        D: Distances (scores de similarité)
        I: Indices des documents correspondants
    """
    q = np.array([query_vec], dtype="float32")
    faiss.normalize_L2(q)
    D, I = index.search(q, k)  # distances, indices
    return D[0], I[0]

# Test des requêtes
print("🔍 Résultats de recherche FAISS:\n")
print("=" * 80)

for qi, qv in enumerate(Q):
    D, I = faiss_search(qv, k=2)
    print(f"\n❓ Query: {queries[qi]}")
    print("-" * 80)
    for rank, (score, idx) in enumerate(zip(D, I), 1):
        print(f"  {rank}. [{docs[idx]['id']}] score={round(float(score), 4)}")
        print(f"     → {docs[idx]['text']}")

🔍 Résultats de recherche FAISS:


❓ Query: How do agents use memory?
--------------------------------------------------------------------------------
  1. [d1] score=0.6091
     → Agentic AI agents use tools, memory, and goals to act.
  2. [d2] score=0.3129
     → LangChain and CrewAI help orchestrate multi-agent workflows.

❓ Query: Name a framework for multi-agent orchestration.
--------------------------------------------------------------------------------
  1. [d2] score=0.5671
     → LangChain and CrewAI help orchestrate multi-agent workflows.
  2. [d1] score=0.3924
     → Agentic AI agents use tools, memory, and goals to act.

❓ Query: Why is RAG useful?
--------------------------------------------------------------------------------
  1. [d3] score=0.6225
     → RAG retrieves external knowledge to improve answer accuracy.
  2. [d5] score=0.3436
     → Planning loops and ReAct improve reasoning in complex tasks.


### 4c. Sauvegarde et Chargement de l'index

In [8]:
# Sauvegarde de l'index
index_file = "faiss_agentic.index"
faiss.write_index(index, index_file)
print(f"💾 Index sauvegardé: {index_file}")

# Rechargement de l'index
index2 = faiss.read_index(index_file)
print(f"📂 Index rechargé: {index2.ntotal} vecteurs")

# Vérification que l'index rechargé fonctionne
test_query = Q[0]
q_test = np.array([test_query], dtype="float32")
faiss.normalize_L2(q_test)
D_test, I_test = index2.search(q_test, 1)
print(f"\n✅ Test de l'index rechargé:")
print(f"   Meilleur résultat: [{docs[I_test[0][0]]['id']}] score={round(float(D_test[0][0]), 4)}")

💾 Index sauvegardé: faiss_agentic.index
📂 Index rechargé: 5 vecteurs

✅ Test de l'index rechargé:
   Meilleur résultat: [d1] score=0.6091


### 4d. Mapping des Métadonnées

⚠️ **Important:** FAISS ne stocke que les vecteurs. Vous devez maintenir un mapping séparé ID→métadonnées.

In [9]:
import json
import pickle

# Création du mapping index → document
id_map = {i: d for i, d in enumerate(docs)}

print("📋 Mapping ID → Métadonnées:")
for idx, doc in id_map.items():
    print(f"  Index {idx} → Document '{doc['id']}'")

# Sauvegarde du mapping (JSON)
with open("faiss_metadata.json", "w") as f:
    json.dump(id_map, f, indent=2)
print("\n💾 Métadonnées sauvegardées: faiss_metadata.json")

# Alternative: sauvegarde en pickle (plus rapide pour gros volumes)
with open("faiss_metadata.pkl", "wb") as f:
    pickle.dump(id_map, f)
print("💾 Métadonnées sauvegardées: faiss_metadata.pkl")

# Rechargement
with open("faiss_metadata.json", "r") as f:
    id_map_loaded = json.load(f)
    # Conversion des clés de string à int
    id_map_loaded = {int(k): v for k, v in id_map_loaded.items()}

print(f"\n✅ Métadonnées rechargées: {len(id_map_loaded)} entrées")

📋 Mapping ID → Métadonnées:
  Index 0 → Document 'd1'
  Index 1 → Document 'd2'
  Index 2 → Document 'd3'
  Index 3 → Document 'd4'
  Index 4 → Document 'd5'

💾 Métadonnées sauvegardées: faiss_metadata.json
💾 Métadonnées sauvegardées: faiss_metadata.pkl

✅ Métadonnées rechargées: 5 entrées


## 5. Pinecone: Création d'Index, Upsert, Query (Managé)

Pinecone est une base vectorielle managée optimisée pour la production.

### Initialisation et création de l'Index

In [11]:
import time
from pinecone import Pinecone, ServerlessSpec

# Initialisation du client Pinecone (v3)
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = os.getenv("PINECONE_INDEX", "agentic-ai-lab")

print(f"🔗 Connexion à Pinecone...")
print(f"📇 Index cible: {index_name}\n")

# Liste des index existants
existing_indexes = [i["name"] for i in pc.list_indexes()]
print(f"📋 Index existants: {existing_indexes}")

# Création de l'index s'il n'existe pas
if index_name not in existing_indexes:
    print(f"\n🏗️ Création de l'index '{index_name}'...")
    pc.create_index(
        name=index_name,
        dimension=dim,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

    # Attente que l'index soit prêt
    print("⏳ Attente de l'initialisation de l'index...")
    while True:
        desc = pc.describe_index(index_name)
        if desc.status["ready"]:
            print("✅ Index prêt!")
            break
        print("   Toujours en cours...")
        time.sleep(2)
else:
    print(f"✅ Index '{index_name}' existe déjà")

# Connexion à l'index
index_pinecone = pc.Index(index_name)
stats = index_pinecone.describe_index_stats()
print(f"\n📊 Statistiques de l'index:")
print(f"   - Vecteurs: {stats['total_vector_count']}")
print(f"   - Dimension: {stats.get('dimension', 'N/A')}")

🔗 Connexion à Pinecone...
📇 Index cible: agentic-ai-lab

📋 Index existants: ['agentic-ai-lab']
✅ Index 'agentic-ai-lab' existe déjà

📊 Statistiques de l'index:
   - Vecteurs: 0
   - Dimension: 1536


### 5a. Upsert des Vecteurs avec Métadonnées

In [12]:
def normalize(v):
    """Normalise un vecteur (optionnel pour Pinecone cosine, mais recommandé pour la cohérence."""
    v = np.array(v, dtype="float32")
    n = np.linalg.norm(v)
    return (v / n).tolist() if n > 0 else v.tolist()

# Préparation des vecteurs avec métadonnées
vectors = [
    {
        "id": d["id"],
        "values": normalize(vec),
        "metadata": {"text": d["text"]}
    }
    for d, vec in zip(docs, X)
]

print(f"📤 Upsert de {len(vectors)} vecteurs dans Pinecone...\n")

# Upsert dans Pinecone
upsert_response = index_pinecone.upsert(vectors=vectors)

print(f"✅ Upsert terminé")
print(f"   - Vecteurs insérés: {upsert_response['upserted_count']}")

# Attente de la synchronisation
time.sleep(2)

# Vérification
stats = index_pinecone.describe_index_stats()
print(f"\n📊 Statistiques mises à jour:")
print(f"   - Total vecteurs: {stats['total_vector_count']}")

📤 Upsert de 5 vecteurs dans Pinecone...

✅ Upsert terminé
   - Vecteurs insérés: 5

📊 Statistiques mises à jour:
   - Total vecteurs: 5


### 5b. Recherche dans Pinecone

In [13]:
def pinecone_search(query_vec, top_k=3):
    """
    Recherche les k vecteurs les plus similaires dans Pinecone.

    Args:
        query_vec: Vecteur de requête
        top_k: Nombre de résultats à retourner

    Returns:
        Résultats de la requête Pinecone
    """
    res = index_pinecone.query(
        vector=normalize(query_vec),
        top_k=top_k,
        include_metadata=True
    )
    return res

# Test des requêtes
print("🔍 Résultats de recherche Pinecone:\n")
print("=" * 80)

for qi, qv in enumerate(Q):
    res = pinecone_search(qv, top_k=2)
    print(f"\n❓ Query: {queries[qi]}")
    print("-" * 80)

    for rank, match in enumerate(res["matches"], 1):
        print(f"  {rank}. [{match['id']}] score={round(match['score'], 4)}")
        print(f"     → {match['metadata']['text']}")

🔍 Résultats de recherche Pinecone:


❓ Query: How do agents use memory?
--------------------------------------------------------------------------------
  1. [d1] score=0.6094
     → Agentic AI agents use tools, memory, and goals to act.
  2. [d2] score=0.313
     → LangChain and CrewAI help orchestrate multi-agent workflows.

❓ Query: Name a framework for multi-agent orchestration.
--------------------------------------------------------------------------------
  1. [d2] score=0.5674
     → LangChain and CrewAI help orchestrate multi-agent workflows.
  2. [d1] score=0.3923
     → Agentic AI agents use tools, memory, and goals to act.

❓ Query: Why is RAG useful?
--------------------------------------------------------------------------------
  1. [d3] score=0.6228
     → RAG retrieves external knowledge to improve answer accuracy.
  2. [d5] score=0.344
     → Planning loops and ReAct improve reasoning in complex tasks.


### 5c. (Optionnel) Suppression et Nettoyage

In [ ]:
# ⚠️ ATTENTION: Décommenter uniquement si vous voulez supprimer des données

# # Suppression d'un seul vecteur
# print("🗑️ Suppression du vecteur 'd5'...")
# index_pinecone.delete(ids=["d5"])
# print("✅ Vecteur supprimé")

# # Suppression de TOUS les vecteurs de l'index (mais garde l'index)
# print("🗑️ Suppression de TOUS les vecteurs...")
# index_pinecone.delete(delete_all=True)
# print("✅ Index vidé")

# # Suppression complète de l'index (⚠️ DANGER!)
# # print(f"🗑️ SUPPRESSION DE L'INDEX '{index_name}'...")
# # pc.delete_index(index_name)
# # print("✅ Index supprimé")

print("ℹ️ Section cleanup - décommenter le code pour activer les suppressions")

## 6. (Optionnel) Intégration LangChain avec Pinecone

Si vous prévoyez d'utiliser des chaînes/agents LangChain, voici comment intégrer Pinecone comme retriever.

In [14]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

print("🔗 Initialisation du retriever LangChain avec Pinecone...\n")

# Création de l'objet embeddings LangChain
lc_embeddings = OpenAIEmbeddings(model=EMB_MODEL)

# Création du vector store LangChain
store = PineconeVectorStore(
    index_name=index_name,
    embedding=lc_embeddings
)

print("✅ Vector store LangChain créé\n")

# Test de recherche avec LangChain
print("🔍 Test de recherche LangChain:\n")
print("=" * 80)

test_query = "How do agents use memory?"
print(f"❓ Query: {test_query}\n")

docs_found = store.similarity_search(test_query, k=2)

for i, doc in enumerate(docs_found, 1):
    print(f"{i}. {doc.page_content}")
    if doc.metadata:
        print(f"   Metadata: {doc.metadata}")

print("\n✅ Intégration LangChain opérationnelle")

🔗 Initialisation du retriever LangChain avec Pinecone...

✅ Vector store LangChain créé

🔍 Test de recherche LangChain:

❓ Query: How do agents use memory?

1. Agentic AI agents use tools, memory, and goals to act.
2. LangChain and CrewAI help orchestrate multi-agent workflows.

✅ Intégration LangChain opérationnelle


## 7. Expérimentations et Prochaines Étapes

### 🧪 Que tester maintenant?

1. **Métriques de distance:**
   - Comparer cosine ↔ dot product (ajuster la normalisation)
   - Tester avec/sans normalisation sur Pinecone

2. **Performance et scalabilité:**
   - Augmenter à plusieurs milliers de documents
   - Mesurer la latence FAISS vs Pinecone
   - Tester différents types d'index FAISS (IVF, HNSW)

3. **Métadonnées enrichies:**
   - Ajouter: titre, URL, tags, timestamp, catégorie
   - Utiliser les filtres de métadonnées dans Pinecone
   - Exemple: `filter={"category": "technical"}`

4. **Intégration RAG:**
   - Connecter ce retriever à une chaîne RAG (Semaine 9)
   - Construire un agent avec accès à cette base de connaissances
   - Implémenter un système de question-réponse complet

5. **Corpus personnalisé:**
   - Remplacer les documents exemple par vos propres données
   - Charger depuis des fichiers (PDF, TXT, JSON)
   - Implémenter du chunking pour documents longs

### 📚 Ressources supplémentaires

- [FAISS Documentation](https://github.com/facebookresearch/faiss/wiki)
- [Pinecone Documentation](https://docs.pinecone.io/)
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)
- [LangChain Vector Stores](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

### Cellule d'expérimentation libre

Utilisez cette cellule pour vos propres tests et expérimentations.

In [ ]:
# 🧪 Zone d'expérimentation
# Testez vos propres requêtes et documents ici

# Exemple: Test avec une nouvelle requête
# new_query = "Your question here"
# new_query_vec = get_embedding(new_query)

# # FAISS
# D, I = faiss_search(new_query_vec, k=3)
# print("FAISS Results:")
# for score, idx in zip(D, I):
#     print(f"  {docs[idx]['id']}: {docs[idx]['text']} (score: {score:.4f})")

# # Pinecone
# results = pinecone_search(new_query_vec, top_k=3)
# print("\nPinecone Results:")
# for match in results['matches']:
#     print(f"  {match['id']}: {match['metadata']['text']} (score: {match['score']:.4f})")

print("💡 Décommentez le code ci-dessus pour tester vos propres requêtes")

## 📋 Récapitulatif

### ✅ Ce que vous avez accompli:

1. ✅ Installation et configuration de l'environnement (OpenAI + Pinecone)
2. ✅ Génération d'embeddings avec OpenAI `text-embedding-3-small`
3. ✅ Construction et interrogation d'un index FAISS local
4. ✅ Persistance et rechargement d'index FAISS
5. ✅ Gestion des métadonnées pour FAISS
6. ✅ Création et configuration d'un index Pinecone
7. ✅ Upsert de vecteurs avec métadonnées dans Pinecone
8. ✅ Recherche de similarité dans Pinecone
9. ✅ Intégration avec LangChain

### 🎯 Prochaines étapes:

- Expérimenter avec vos propres données
- Optimiser les performances pour des corpus plus larges
- Intégrer dans un système RAG complet
- Explorer les filtres de métadonnées avancés

### 💾 Fichiers générés:

- `faiss_agentic.index` - Index FAISS persisté
- `faiss_metadata.json` - Métadonnées des documents (JSON)
- `faiss_metadata.pkl` - Métadonnées des documents (Pickle)

---

**🎉 Félicitations! Vous avez maintenant un workflow local FAISS pour le prototypage rapide et un index Pinecone managé prêt pour la production!**